<a href="https://colab.research.google.com/github/Nischal5123/foreCache-interaction/blob/develop/iteractionsDatatExploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
#global path
global_drive_path="/content/drive/MyDrive/ForeCache/interactions/"
          

In [47]:
import os
import pandas as pd
import json
from pandas.io.json import json_normalize
from tqdm import tqdm


#for reward
import time
import datetime

pd.options.mode.chained_assignment = None  # default='warn'


In [48]:
def clean_files(filePath,fileName):

  lastline = None
 # filePath="/content/drive/Shareddrives/Colearning-Exploration/Dataset/forecache_interactions_2016/forecache-interaction-analysis/forecache-interaction-analysis/interactions/taskname_ndsi-2d-subtask_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.txt"
  with open(filePath,"r") as f:
      lineList = f.readlines()
      lastline=lineList[-1]
  with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-ndsi-2d-task/"+(str(fileName)+".json"),"w") as g:
  #with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-ndsi-3d-task/"+(str(fileName)+".json"),"w") as g:
  #os.makedirs(os.path.dirname(global_drive_path+"cleanInteractions/cleanInteractions-rgb-hurricane-3d-task/"+(str(fileName)+".json")))
  #with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-rgb-hurricane-3d-task/"+(str(fileName)+".json"),"w+") as g:
   
      for i,line in enumerate(f,0):
          if i == 0:
              line = "["+str(line)+","
              g.write(line)
          elif line == lastline:            
              g.write(line)
              g.write("]")
          else:
              line = str(line)+","
              g.write(line)
          

In [49]:
directory =(str(global_drive_path) +"/ndsi-2d-task")
#directory =(str(global_drive_path) +"/ndsi-3d-task")
#directory =(str(global_drive_path) +"/rgb-hurricane-3d-task")
failed_files=[]
for filename in os.listdir(directory):
  try:
    if filename.endswith(".txt"):
        
        filePath=(os.path.join(directory, filename))
        print(filename)
        name=os.path.splitext(filename)[0]
        clean_files(filePath,name)
    else:
        continue
  except OSError as err:
    print("OS error: {0}".format(err))
    failed_files.append(filename)
  except UnicodeDecodeError:
    print("Unicode Error")
    failed_files.append(filename)
  except BaseException as err:
    failed_files.append(filename)
    
    raise


taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.txt
taskname_ndsi-2d-task_userid_ac5b65d7-f260-4534-9fae-d998b726ed32.txt
taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.txt
taskname_ndsi-2d-task_userid_da52acdd-3cea-483c-86e5-2c2740d371aa.txt
taskname_ndsi-2d-task_userid_d6555293-35c7-4c27-b522-38d67d2d1b1a.txt
taskname_ndsi-2d-task_userid_ff56863b-0710-4a58-ad22-4bf2889c9bc0.txt
taskname_ndsi-2d-task_userid_bc9f9425-e867-4876-a5d9-791229916767.txt
taskname_ndsi-2d-task_userid_bda49380-37ad-41c5-a109-7fa198a7691a.txt
taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.txt
taskname_ndsi-2d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.txt
taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.txt
taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.txt
taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.txt
taskname_ndsi-2d-task_userid_e4221e48-0930-40db-af3c-f4de1ed87582.txt
taskname_ndsi-2d-tas

In [50]:
# directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-ndsi-3d-task")
directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-ndsi-2d-task")
#directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-rgb-hurricane-3d-task")
fileNames=[]
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        filePath=(os.path.join(directory, filename))
        fileNames.append(filename)
    else:
        continue

In [51]:
import numpy as np
import matplotlib.pyplot as plt
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

In [52]:
def get_Angle(df):
  interactionData= pd.json_normalize(df.interactionData)
  x1=[]
  x2=[]
  y1=[]
  y2=[]
  angle=[]
  for i in range(len(interactionData['interactionType'])):
    #handles if the interactiontype doesn't have a coordinate : if interactionType is zoom
    if ((isinstance(interactionData['state.dragStart'][i], list) or isinstance(interactionData['state.dragEnd'][i], list) )==False):
      
      interactionData['state.dragStart'][i]=[0,0]
      interactionData['state.dragEnd'][i]=[0,0]
    #get between drag start and drag end
    angle.append(angle_between(interactionData['state.dragStart'][i],interactionData['state.dragEnd'][i]))
    #print(angle)

    #this are to plot later on
    x1.append(interactionData['state.dragStart'][i][0])
    y1.append(interactionData['state.dragStart'][i][1])

    x2.append(interactionData['state.dragEnd'][i][0])
    y2.append(interactionData['state.dragEnd'][i][1])
  return angle



In [60]:
#to generate reward based on radiclick and snapshot data 
def get_Reward(dataframe,filenamed):
  #global path
  reward=[]
  subtask=[]
  raw_timestamp= dataframe.timestampMillis
  snapshot_global_drive_path="/content/drive/MyDrive/ForeCache/forecache_user_study_data_dump/forecache_user_study_data_dump"
  #get a pandas dataframe object from json file
  path= snapshot_global_drive_path + "/snapshots.json"
  with open(path) as project_file:    
      data = json.load(project_file) 
  df_snapshots = pd.json_normalize(data)

  #get userid from fileName
  #filenamed[filenamed.find("userid_")+7: filenamed.find(".csv")] need to remove  hard code later
  dataframe_dimension=df_snapshots.loc[(df_snapshots['taskname']=="ndsi-2d-task") & (df_snapshots['userid']==filenamed[filenamed.find("userid_")+7: filenamed.find(".json")])]
  print("There are ", len(dataframe_dimension) , "Snapshot time")
  for i in tqdm(range(len(raw_timestamp))):
      stringmyDate=str(raw_timestamp[i])
      myDate=stringmyDate[: stringmyDate.find(".")]
      dt = datetime.datetime.strptime(myDate, "%Y-%m-%d %H:%M:%S")
      a=time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0)
      temp=0
      task=1
      for j in range(len(dataframe_dimension["timestampMillis"])):
        # print("data",dataframe_dimension.timestampMillis.iloc[j])
        # print("a",a)
        # print(abs(a - (dataframe_dimension.timestampMillis.iloc[j]/1000)))
        if abs(a - (dataframe_dimension.timestampMillis.iloc[j]/1000)) < 10:
          temp=temp+10
        else:
         temp=temp+0
        if (a - (dataframe_dimension.timestampMillis.iloc[j]/1000)) < 0:
          task=task
        else:
          task=task+1
      subtask.append(task)
      reward.append(temp)
  return reward, subtask
    

In [54]:
def get_Zoom(df_zoom):
  df_zoom['state.newZoom'] = df_zoom['state.newZoom'].replace(np.nan, 0)
  lastZoom=[0,0]
  zoom_level=[]
  for i in tqdm(range(len(df_zoom))):
    if df_zoom['state.newZoom'][i]==0:
      if (lastZoom!=[0,0]):
          df_zoom['state.newZoom'][i]=lastZoom
      else:
        df_zoom['state.newZoom'][i]= [0,0]
        lastZoom= [0,0]
    else:
      lastZoom = df_zoom['state.newZoom'][i]
    zoom_level.append(df_zoom['state.newZoom'][i][0])
  return zoom_level

In [58]:
def get_IdleTime(df):
  idle_time=[]
  previous_time=df['timestampMillis'][0]
  for i in tqdm(range(len(df))):
      wait_time=df['timestampMillis'][i]-previous_time
      previous_time=df['timestampMillis'][i]


      #fixing format

      stringmyDate=str(wait_time)
      myDate=stringmyDate[: stringmyDate.find(".")]
      dt = datetime.datetime.strptime(myDate, "%Y-%m-%d %H:%M:%S")
      wait_time=time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0)
      idle_time.append(wait_time)
  return idle_time

In [56]:
def get_State(df_state):
  states_array=[]
  for i in range(len(df_state)):
    if pd.isna(df_state.phase[i]):
      states_array.append('Answering')
    else:
      states_array.append(df_state.phase[i])
  return states_array

In [59]:
error_files=[]

for filename in tqdm(fileNames):
  try:
    saveDir=(str(global_drive_path) +"/stateActionModel/ndsi-2d")
    # saveDir=(str(global_drive_path) +"/stateActionModel/ndsi-3d")
    
    #saveDir=(str(global_drive_path) +"/stateActionModel/rgb-hurricane-3d")
    #get a pandas dataframe object from json file
    df = pd.read_json(os.path.join(directory,filename))
    print("Read file:",filename)
    interactionData=pd.json_normalize(df.interactionData)
    final_dataframe = pd.DataFrame(columns=['Action','State','TimeStamp','IdleTime','Reward','Subtask','ZoomLevel'])
    final_dataframe['Action']=interactionData.interactionType
    final_dataframe['State']=get_State(df)
    final_dataframe['TimeStamp']=df.timestampMillis
    final_dataframe['IdleTime']=get_IdleTime(df)
    final_dataframe['Reward'],final_dataframe['Subtask']= get_Reward(df,filename)
    final_dataframe['Angle']= get_Angle(df)
    final_dataframe['ZoomLevel']= get_Zoom(interactionData)	
    txt_version= filename.replace("json","csv")
    final_dataframe.to_csv(os.path.join(saveDir,txt_version), index=False)
  except AttributeError:
    print("AttributeError")
    error_files.append(filename)


  0%|          | 0/20 [00:00<?, ?it/s]

Read file: taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.json



  0%|          | 0/20 [00:00<?, ?it/s]


ValueError: ignored

In [ ]:
path=(str(global_drive_path) +"/stateActionModel/ndsi-2d")
fileNames_NDSI_2D=[]
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filePath=(os.path.join(directory, filename))
        fileNames_NDSI_2D.append(filename)
    else:
        continue

In [ ]:
path=(str(global_drive_path) +"/stateActionModel/ndsi-3d")
fileNames_NDSI_3D=[]
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filePath=(os.path.join(directory, filename))
        fileNames_NDSI_3D.append(filename)
    else:
        continue

In [ ]:
#all 2d tasks
fileNames_NDSI_2D.remove

['taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
 'taskname_ndsi-2d-task_userid_ac5b65d7-f260-4534-9fae-d998b726ed32.csv',
 'taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv',
 'taskname_ndsi-2d-task_userid_da52acdd-3cea-483c-86e5-2c2740d371aa.csv',
 'taskname_ndsi-2d-task_userid_d6555293-35c7-4c27-b522-38d67d2d1b1a.csv',
 'taskname_ndsi-2d-task_userid_ff56863b-0710-4a58-ad22-4bf2889c9bc0.csv',
 'taskname_ndsi-2d-task_userid_bc9f9425-e867-4876-a5d9-791229916767.csv',
 'taskname_ndsi-2d-task_userid_bda49380-37ad-41c5-a109-7fa198a7691a.csv',
 'taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv',
 'taskname_ndsi-2d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.csv',
 'taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'taskname_ndsi-2d-task_userid_cd3ef507-e3e9-4edb-9222-90a69a9bf272.csv',
 'taskname_ndsi-2d-task_userid_93be0a02-3f7a-46c9-b526-aa956ed00856.csv',
 'taskname_ndsi-2d-task_userid_6d49fab

In [ ]:
#all 3d tasks
fileNames_NDSI_3D

['taskname_ndsi-3d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.csv',
 'taskname_ndsi-3d-task_userid_bda49380-37ad-41c5-a109-7fa198a7691a.csv',
 'taskname_ndsi-3d-task_userid_954edb7c-4eae-47ab-9338-5c5c7eccac2d.csv',
 'taskname_ndsi-3d-task_userid_e4221e48-0930-40db-af3c-f4de1ed87582.csv',
 'taskname_ndsi-3d-task_userid_d6555293-35c7-4c27-b522-38d67d2d1b1a.csv',
 'taskname_ndsi-3d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.csv',
 'taskname_ndsi-3d-task_userid_c7cac3a0-2f18-4359-905e-ce343d6d92f1.csv',
 'taskname_ndsi-3d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv',
 'taskname_ndsi-3d-task_userid_ac5b65d7-f260-4534-9fae-d998b726ed32.csv',
 'taskname_ndsi-3d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'taskname_ndsi-3d-task_userid_ff56863b-0710-4a58-ad22-4bf2889c9bc0.csv',
 'taskname_ndsi-3d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv',
 'taskname_ndsi-3d-task_userid_a6aab5f5-fdb6-41df-9fc6-221d70f8c6e8.csv',
 'taskname_ndsi-3d-task_userid_93be0a0

In [ ]:
matchingName=[]
for i in range(len(fileNames_NDSI_2D)):
  for j in range(len(fileNames_NDSI_2D)):
  # if fileNames_NDSI_2D[i].removesuffix('taskname_ndsi-2d-task_') == fileNames_NDSI_3D[i].removesuffix('taskname_ndsi-3d-task_'):
   if fileNames_NDSI_2D[i][22:] == fileNames_NDSI_3D[j][22:]:
      matchingName.append(fileNames_NDSI_2D[i][22:])
      print(fileNames_NDSI_3D[i])

taskname_ndsi-3d-task_userid_bda49380-37ad-41c5-a109-7fa198a7691a.csv
taskname_ndsi-3d-task_userid_954edb7c-4eae-47ab-9338-5c5c7eccac2d.csv
taskname_ndsi-3d-task_userid_e4221e48-0930-40db-af3c-f4de1ed87582.csv
taskname_ndsi-3d-task_userid_d6555293-35c7-4c27-b522-38d67d2d1b1a.csv
taskname_ndsi-3d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.csv
taskname_ndsi-3d-task_userid_c7cac3a0-2f18-4359-905e-ce343d6d92f1.csv
taskname_ndsi-3d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv
taskname_ndsi-3d-task_userid_ac5b65d7-f260-4534-9fae-d998b726ed32.csv
taskname_ndsi-3d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv
taskname_ndsi-3d-task_userid_ff56863b-0710-4a58-ad22-4bf2889c9bc0.csv
taskname_ndsi-3d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv
taskname_ndsi-3d-task_userid_a6aab5f5-fdb6-41df-9fc6-221d70f8c6e8.csv
taskname_ndsi-3d-task_userid_93be0a02-3f7a-46c9-b526-aa956ed00856.csv
taskname_ndsi-3d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv
taskname_ndsi-3d-tas

In [ ]:
len(matchingName) # so 19 users are the same 

19

In [ ]:
filenamed="taskname_ndsi-3d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.csv"

In [ ]:
filenamed[filenamed.find("userid_")+7: filenamed.find(".csv")]

'8b544d24-3274-4bb0-9719-fd2bccc87b02'

In [ ]:
df.head(5)

,phase,toFetch,timestampMillis,interactionData
0,Foraging,"[{'dimindices': [0, 0, 1], 'zoom': [0, 0], 'na...",2016-06-15 19:57:43.550,"{'state': {}, 'viewName': None, 'interactionTy..."
1,Foraging,"[{'dimindices': [1, 0, 0], 'zoom': [0, 0], 'na...",2016-06-15 19:58:14.420,"{'state': {'dragEnd': [253, 164], 'dragStart':..."
2,Navigation,"[{'dimindices': [0, 0, 0], 'zoom': [1, 0], 'na...",2016-06-15 19:58:15.182,"{'state': {'newMids': {'1': 79.77037037037039,..."
3,Navigation,[],2016-06-15 19:58:19.564,"{'state': {'dragEnd': [397, 174], 'dragStart':..."
4,Navigation,"[{'dimindices': [2, 1, 0], 'zoom': [2, 0], 'na...",2016-06-15 19:58:20.211,{'state': {'newMids': {'1': 158.20740740740746...


In [ ]:
df.phase

0         Foraging
1         Foraging
2       Navigation
3       Navigation
4       Navigation
          ...     
241     Navigation
242    Sensemaking
243    Sensemaking
244    Sensemaking
245    Sensemaking
Name: phase, Length: 246, dtype: object

In [ ]:
interactionData= pd.io.json.json_normalize(df.interactionData)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [ ]:
interactionData.interactionType

0     start
1      zoom
2       pan
3       pan
4       pan
5       pan
6      zoom
7      zoom
8      zoom
9      zoom
10     zoom
11     zoom
12     zoom
13     zoom
14      pan
15     zoom
16      pan
17     zoom
18     zoom
19      pan
20     zoom
21      pan
22     zoom
23      pan
24      pan
25     zoom
26     zoom
27     zoom
28      pan
29     zoom
30      pan
31     zoom
32     zoom
33     zoom
34      pan
35      pan
36      pan
37      pan
38      pan
39      pan
40     zoom
41     zoom
42     zoom
43     zoom
44      pan
45      pan
46      pan
47     zoom
48     zoom
49      pan
50     zoom
51     zoom
52      pan
53     zoom
54      pan
55     zoom
Name: interactionType, dtype: object

In [ ]:
final_dataframe = pd.DataFrame(columns=['Action','State','TimeStamp'])
final_dataframe['Action']=interactionData.interactionType
final_dataframe['State']=df.phase
final_dataframe['TimeStamp']=df.timestampMillis

In [ ]:
final_dataframe.head

<bound method NDFrame.head of Empty DataFrame
Columns: [Action, State, TimeStamp]
Index: []>

In [ ]:
final_dataframe['Action']=interactionData.interactionType
final_dataframe['State']=df.phase
final_dataframe['TimeStamp']=df.timestampMillis

In [ ]:
final_dataframe

,Action,State,TimeStamp
0,start,Foraging,2016-06-15 16:30:54.682
1,zoom,Foraging,2016-06-15 16:31:54.487
2,pan,Foraging,2016-06-15 16:31:56.208
3,pan,Foraging,2016-06-15 16:31:59.997
4,pan,Foraging,2016-06-15 16:32:08.098
5,pan,Foraging,2016-06-15 16:32:19.471
6,zoom,Navigation,2016-06-15 16:32:44.131
7,zoom,Navigation,2016-06-15 16:32:45.547
8,zoom,Navigation,2016-06-15 16:32:45.987
9,zoom,Navigation,2016-06-15 16:32:46.627


In [ ]:
for item in df.toFetch:
  print (item)

[{'dimindices': [0, 0], 'zoom': [0], 'name': 'zoom_0_pos_0_0'}]
[{'dimindices': [0, 0], 'zoom': [1], 'name': 'zoom_1_pos_0_0'}, {'dimindices': [1, 0], 'zoom': [1], 'name': 'zoom_1_pos_1_0'}, {'dimindices': [0, 1], 'zoom': [1], 'name': 'zoom_1_pos_0_1'}, {'dimindices': [1, 1], 'zoom': [1], 'name': 'zoom_1_pos_1_1'}]
[]
[{'dimindices': [0, 2], 'zoom': [1], 'name': 'zoom_1_pos_0_2'}, {'dimindices': [1, 2], 'zoom': [1], 'name': 'zoom_1_pos_1_2'}]
[{'dimindices': [0, 0], 'zoom': [1], 'name': 'zoom_1_pos_0_0'}, {'dimindices': [1, 0], 'zoom': [1], 'name': 'zoom_1_pos_1_0'}]
[]
[{'dimindices': [0, 2], 'zoom': [2], 'name': 'zoom_2_pos_0_2'}, {'dimindices': [1, 2], 'zoom': [2], 'name': 'zoom_2_pos_1_2'}, {'dimindices': [0, 3], 'zoom': [2], 'name': 'zoom_2_pos_0_3'}, {'dimindices': [1, 3], 'zoom': [2], 'name': 'zoom_2_pos_1_3'}]
[{'dimindices': [1, 4], 'zoom': [3], 'name': 'zoom_3_pos_1_4'}, {'dimindices': [2, 4], 'zoom': [3], 'name': 'zoom_3_pos_2_4'}, {'dimindices': [1, 5], 'zoom': [3], 'name':

In [ ]:
toFetch

[  dimindices zoom            name
 0     [0, 0]  [0]  zoom_0_pos_0_0,   dimindices zoom            name
 0     [0, 0]  [0]  zoom_0_pos_0_0,   dimindices zoom            name
 0     [0, 0]  [0]  zoom_0_pos_0_0,   dimindices zoom            name
 0     [0, 0]  [0]  zoom_0_pos_0_0]

In [ ]:
import json
from pandas.io.json import json_normalize
normalized_json = json_normalize(df['taskname_ndsi-2d-subtask_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45'])
normalized_json.head(3 )


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,phase,toFetch,timestampMillis,interactionData.viewName,interactionData.interactionType,interactionData.state.newMids.1,interactionData.state.newMids.0,interactionData.state.oldMids.1,interactionData.state.oldMids.0,interactionData.state.newZoom,interactionData.state.oldZoom,interactionData.state.dragEnd,interactionData.state.dragStart
0,Foraging,"[{'dimindices': [0, 0], 'zoom': [0], 'name': '...",1466008254682,None,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Foraging,"[{'dimindices': [0, 0], 'zoom': [1], 'name': '...",1466008314487,heatmap,zoom,89.0,179.0,44.5,89.5,[1],[0],NaN,NaN
2,Foraging,[],1466008316208,heatmap,pan,NaN,NaN,NaN,NaN,NaN,NaN,"[360, 202.5]","[283, 71.5]"


In [ ]:
normalized_json['toFetch']

0     [{'dimindices': [0, 0], 'zoom': [0], 'name': '...
1     [{'dimindices': [0, 0], 'zoom': [1], 'name': '...
2                                                    []
3     [{'dimindices': [0, 2], 'zoom': [1], 'name': '...
4     [{'dimindices': [0, 0], 'zoom': [1], 'name': '...
5                                                    []
6     [{'dimindices': [0, 2], 'zoom': [2], 'name': '...
7     [{'dimindices': [1, 4], 'zoom': [3], 'name': '...
8     [{'dimindices': [4, 8], 'zoom': [4], 'name': '...
9     [{'dimindices': [9, 17], 'zoom': [5], 'name': ...
10    [{'dimindices': [4, 8], 'zoom': [4], 'name': '...
11    [{'dimindices': [1, 4], 'zoom': [3], 'name': '...
12    [{'dimindices': [0, 2], 'zoom': [2], 'name': '...
13    [{'dimindices': [0, 0], 'zoom': [1], 'name': '...
14                                                   []
15    [{'dimindices': [0, 2], 'zoom': [2], 'name': '...
16                                                   []
17    [{'dimindices': [1, 4], 'zoom': [3], 'name

In [ ]:
#get a pandas dataframe object from json file
#df = pd.read_json("/content/interactions_data/zoom_0_pos_0_0.json")
import json
import pandas as pd
from pandas.io.json import json_normalize
path= '/content/drive/Shareddrives/Colearning-Exploration/Dataset/forecache_interactions_2016/ndsi_2d_jsons/zoom_0_pos_0_0.json'
with open(path) as project_file:    
    data = json.load(project_file)  

df_zoom = pd.json_normalize(data)
df_zoom.head(5)

,dataTypes,columns,domains,attributes,id.dimindices,id.zoom
0,"[int, int, float, float, int, float, float, fl...","[[13, 14, 19, 20, 20, 21, 21, 21, 21, 22, 22, ...","[[13, 147], [14, 89], [0.40029, 0.953198], [0....","[longitude_e4, latitude_e4, ndsi, total_snow, ...","[0, 0]",[0]


In [ ]:
df_zoom.domains

0    [[13, 147], [14, 89], [0.40029, 0.953198], [0....
Name: domains, dtype: object